In [1]:
import os
import copy
import torch
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader, build_detection_train_loader

In [2]:
# Register Dataset
try:
    register_coco_instances('coco_trash_train', {}, '../dataset/train.json', '../dataset/')
except AssertionError:
    pass

try:
    register_coco_instances('coco_trash_test', {}, '../dataset/test.json', '../dataset/')
except AssertionError:
    pass

MetadataCatalog.get('coco_trash_train').thing_classes = ["General trash", "Paper", "Paper pack", "Metal", 
                                                         "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

In [3]:
# config 불러오기
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'))

In [4]:
# config 수정하기
cfg.DATASETS.TRAIN = ('coco_trash_train',)
cfg.DATASETS.TEST = ('coco_trash_test',)

cfg.DATALOADER.NUM_WOREKRS = 2

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml')

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 15000
cfg.SOLVER.STEPS = (8000,12000)
cfg.SOLVER.GAMMA = 0.005
cfg.SOLVER.CHECKPOINT_PERIOD = 3000

cfg.OUTPUT_DIR = './output'

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10

cfg.TEST.EVAL_PERIOD = 3000

In [5]:
# mapper - input data를 어떤 형식으로 return할지 (따라서 augmnentation 등 데이터 전처리 포함 됨)
import detectron2.data.transforms as T

def MyMapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    transform_list = [
        T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
        T.RandomBrightness(0.8, 1.8),
        T.RandomContrast(0.6, 1.3)
    ]
    
    image, transforms = T.apply_transform_gens(transform_list, image)
    
    dataset_dict['image'] = torch.as_tensor(image.transpose(2,0,1).astype('float32'))
    
    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop('annotations')
        if obj.get('iscrowd', 0) == 0
    ]
    
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict['instances'] = utils.filter_empty_instances(instances)
    
    return dataset_dict

In [6]:
# trainer - DefaultTrainer를 상속
class MyTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
        cfg, mapper = MyMapper, sampler = sampler
        )
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs('./output_eval', exist_ok = True)
            output_folder = './output_eval'
            
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
# train
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[09/16 14:17:27 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[09/16 14:17:27 d2.data.datasets.coco]: Loaded 4883 images in COCO format from ../dataset/train.json
[09/16 14:17:27 d2.data.build]: Removed 0 images with no usable annotations. 4883 images left.
[09/16 14:17:28 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |  category   | #instances   |  category  | #instances   |
|:-------------:|:-------------|:-----------:|:-------------|:----------:|:-------------|
| General trash | 3966         |    Paper    | 6352         | Paper pack | 897          |
|     Metal     | 936          |    Glass    | 982          |  Plastic   | 2943         |
|   Styrofoam   | 1263         | Plastic bag | 5178         |  Battery   | 159          |
|   Clothing    | 468          |             |              |            |              |
|     total     | 23144        |             |              |            |              |
[09/16 14:17:28 d2.data.build]: Using training sampler TrainingSampler
[09/16 14:17:28 d2.d

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

[09/16 14:17:28 d2.engine.train_loop]: Starting training from iteration 0


/opt/ml/detection/detectron2/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


[09/16 14:17:48 d2.utils.events]:  eta: 3:45:33  iter: 19  total_loss: 3.267  loss_cls: 2.437  loss_box_reg: 0.7491  loss_rpn_cls: 0.05972  loss_rpn_loc: 0.02993  time: 0.9080  data_time: 0.0450  lr: 1.9981e-05  max_mem: 6307M
[09/16 14:18:00 d2.utils.events]:  eta: 3:01:15  iter: 39  total_loss: 2.957  loss_cls: 2.018  loss_box_reg: 0.7535  loss_rpn_cls: 0.07661  loss_rpn_loc: 0.02671  time: 0.7498  data_time: 0.0133  lr: 3.9961e-05  max_mem: 6307M
[09/16 14:18:11 d2.utils.events]:  eta: 2:25:51  iter: 59  total_loss: 2.218  loss_cls: 1.266  loss_box_reg: 0.6683  loss_rpn_cls: 0.1095  loss_rpn_loc: 0.04988  time: 0.6913  data_time: 0.0120  lr: 5.9941e-05  max_mem: 6307M
[09/16 14:18:23 d2.utils.events]:  eta: 2:24:45  iter: 79  total_loss: 1.727  loss_cls: 0.8106  loss_box_reg: 0.6945  loss_rpn_cls: 0.06041  loss_rpn_loc: 0.03994  time: 0.6633  data_time: 0.0119  lr: 7.9921e-05  max_mem: 6307M
[09/16 14:18:35 d2.utils.events]:  eta: 2:24:25  iter: 99  total_loss: 1.556  loss_cls: 0.74

[09/16 14:24:36 d2.utils.events]:  eta: 2:18:09  iter: 719  total_loss: 0.8548  loss_cls: 0.5145  loss_box_reg: 0.3295  loss_rpn_cls: 0.02572  loss_rpn_loc: 0.03155  time: 0.5908  data_time: 0.0121  lr: 0.00071928  max_mem: 6307M
[09/16 14:24:48 d2.utils.events]:  eta: 2:17:57  iter: 739  total_loss: 0.8598  loss_cls: 0.4665  loss_box_reg: 0.331  loss_rpn_cls: 0.02764  loss_rpn_loc: 0.02255  time: 0.5906  data_time: 0.0124  lr: 0.00073926  max_mem: 6307M
[09/16 14:24:59 d2.utils.events]:  eta: 2:17:45  iter: 759  total_loss: 0.9492  loss_cls: 0.4889  loss_box_reg: 0.3392  loss_rpn_cls: 0.0279  loss_rpn_loc: 0.03219  time: 0.5903  data_time: 0.0129  lr: 0.00075924  max_mem: 6307M
[09/16 14:25:11 d2.utils.events]:  eta: 2:17:33  iter: 779  total_loss: 0.8715  loss_cls: 0.4614  loss_box_reg: 0.2897  loss_rpn_cls: 0.0339  loss_rpn_loc: 0.03087  time: 0.5901  data_time: 0.0128  lr: 0.00077922  max_mem: 6307M
[09/16 14:25:23 d2.utils.events]:  eta: 2:17:21  iter: 799  total_loss: 0.8756  los

[09/16 14:31:24 d2.utils.events]:  eta: 2:11:18  iter: 1419  total_loss: 0.6155  loss_cls: 0.3625  loss_box_reg: 0.2718  loss_rpn_cls: 0.01124  loss_rpn_loc: 0.01384  time: 0.5864  data_time: 0.0124  lr: 0.001  max_mem: 6307M
[09/16 14:31:35 d2.utils.events]:  eta: 2:11:06  iter: 1439  total_loss: 0.8386  loss_cls: 0.4307  loss_box_reg: 0.3354  loss_rpn_cls: 0.02291  loss_rpn_loc: 0.02842  time: 0.5864  data_time: 0.0121  lr: 0.001  max_mem: 6307M
[09/16 14:31:47 d2.utils.events]:  eta: 2:10:55  iter: 1459  total_loss: 0.6694  loss_cls: 0.3732  loss_box_reg: 0.2485  loss_rpn_cls: 0.0228  loss_rpn_loc: 0.0182  time: 0.5863  data_time: 0.0124  lr: 0.001  max_mem: 6307M
[09/16 14:31:59 d2.utils.events]:  eta: 2:10:43  iter: 1479  total_loss: 0.7348  loss_cls: 0.4132  loss_box_reg: 0.2766  loss_rpn_cls: 0.01723  loss_rpn_loc: 0.01992  time: 0.5862  data_time: 0.0129  lr: 0.001  max_mem: 6307M
[09/16 14:32:10 d2.utils.events]:  eta: 2:10:32  iter: 1499  total_loss: 0.9484  loss_cls: 0.4847 